In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import logging, os, shutil


import configs
from models import Document
from analysis import product, info, lists, bs, hnr, mns
from tools import check_folder

In [2]:
## 检查文件结构，并创建缺少的文件夹
check_folder(r"./reports/{}".format(configs.report_name))
for folder in ['data', 'image']:
    check_folder(r"./reports/{}/{}".format(configs.report_name, folder))

In [3]:
document = Document()

In [4]:
document.author = configs.author
document.title = '%s-%s%s品类报告'%(info.start_time, info.end_time,info.category_name)
document.foreword = """
这里是前言
"""
document.chapters[0].title = '一、总体情况'
document.chapters[0].foreword = """
{.start_time}至{.end_time}共有{.commodity_total_count}款
商品登榜亚马逊{.category_name}品类的相关榜单，
其中涉及品牌数{.brand_total_count:.0f}个。
截至{.end_time}，登榜商品总评论数达{.review_total_count:.0f}条，
平均星级{.rating_mean:.2f}星。\n\n
榜单的登榜商品及品牌数量情况如下表所示：
""".format(info,info,info,info,info,info,info,info)
document.chapters[0].table = pd.DataFrame(data = {'榜单':list(lists.keys()),
                     '登榜商品数':[rank_list.commodity.count for rank_list in lists.values()],
                     '登榜品牌数':[rank_list.brand.count for rank_list in lists.values()]
                    })

In [5]:
for i in range(3):
    chinese_number = ['一', '二', '三', '四']
    rank_name = list(lists.keys())[i]
    rank_list = lists[rank_name]
    
    chapter = document.chapters[i+1]
    chapter.title = "{}、{}榜单分析：".format(chinese_number[i+1], rank_name)
    
    chapter.chapters[0].title = '1.价格分布'
    chapter.chapters[0].set_image(rank_list.price.image.fig)
    chapter.chapters[0].content = """
    上周 {} 品类 {} 榜单商品最高价${:.2f}，
    最低价${:.2f}，平均价格${:.2f}。{}发现，在上榜的商品中，
    """.format(info.category_name, rank_name, rank_list.price.max, rank_list.price.min, 
              rank_list.price.mean, configs.author)
    
    chapter.chapters[1].title = '2.评论量分布'
    chapter.chapters[1].set_image(rank_list.review.image.fig)
    chapter.chapters[1].content = """
    在上周登榜 {} 品类 {} 榜单的商品中，
    单个商品拥有的最大评论数为{:.2f}条，最小评论数为{:.2f}条，平均评论数{:.2f}条，评论平均星级{:.2f}星。
    """.format(info.category_name, rank_name, rank_list.review.max, rank_list.review.min, 
              rank_list.review.mean, rank_list.review.rating_mean)
    
    chapter.chapters[2].title = '3.商品排行'
    chapter.chapters[2].foreword = '''根据上周 {} 榜单上商品的登榜次数及平均排名，
    {}对登榜的商品进行了排序，其中排名靠前的五款商品如下所示：'''.format(rank_name, configs.author)
    chapter.chapters[2].table = rank_list.commodity.table
    chapter.chapters[2].set_image(rank_list.commodity.image.fig)
    chapter.chapters[2].content = """
    可以看到，在上周{}品类的{}榜单中，
    """.format(info.category_name, rank_name)
    
    chapter.chapters[3].title = '4.品牌排行'
    chapter.chapters[3].table = rank_list.brand.table
    chapter.chapters[3].set_image(rank_list.brand.image.fig)
    chapter.chapters[3].foreword = """根据上周 {} 榜单上各个品牌下商品的登榜次数，{}对登榜的品牌进行了排序，
    其中排名靠前的五个品牌如下所示：""".format(rank_name, configs.author)
    chapter.chapters[3].content = """
    可以看到，在上周{}品类的{}榜单中，
    """.format(info.category_name, rank_name)
    
document.chapters[4].title = '  '
document.chapters[4].content = """
这里是总结
"""

In [6]:
document.save_html("./reports/{}/{}.html".format(configs.report_name, configs.report_name))

NameError: name 'Template' is not defined

In [ ]:
# 把main.ipynb和configs.py备份入报告文件夹
for filename in ['main.ipynb', 'configs.py']:  
    if os.path.exists(r"./reports/{}/{}".format(configs.report_name, filename)):
        logging.warning("文件已添加：./reports/{}/{}".format(configs.report_name, filename))
    shutil.copy(filename, r"./reports/{}/{}".format(configs.report_name, filename))
    logging.info("已复制文件：{}".format(filename))